In [3]:
import time 

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pinocchio as pin
from scipy.optimize import least_squares
from scipy.special import huber
from projectyl.utils.properties import ELBOW, SHOULDER, WRIST
from projectyl.utils.arm import plot_optimization_curves

from projectyl.dynamics.inverse_kinematics import build_arm_model
from projectyl.utils.arm import interactive_replay_sequence
from projectyl.utils.io import Dump
from interactive_pipe.helper import _private
from typing import List, Tuple, Dict, Any
_private.registered_controls_names = [] # this is for notebooks where you re-execute cells everytime.

%load_ext autoreload
%autoreload 2
global_params = {}
build_arm_model(global_params, headless=False, free_elbow=True)
arm_robot = global_params["arm"]
viz = global_params["viz"]  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [4]:
arm_robot.model.createData()
viz.display(arm_robot.q0)

TESTS arm moving, random constant acceleration

In [59]:
data = Dump.load_pickle("../__out/0009_weight_lifting_main_camera_30fps/coarse_ik.pkl")

# INITIAL STATE LIST
q_reduced = np.array(data["q"])
T = 30

elbow_angle_cos = np.cos(q_reduced[:, 4])
elbow_angle_sin = np.sin(q_reduced[:, 4])


q = np.concatenate([q_reduced[:,:4], elbow_angle_cos.reshape(-1, 1), elbow_angle_sin.reshape(-1, 1)], axis=1)

ground_truth_q = q[:T].flatten()

# 3D points
ground_truth_p = np.concatenate([
    np.concatenate([
        data["3dpoints"][member][t].translation for member in [SHOULDER, ELBOW, WRIST]
    ]) 
    for t in range(T)
])

# initialization from IK estimation
initial = np.concatenate([ground_truth_q, np.zeros((T - 4) * arm_robot.model.nv)])
DT = 1/30. # 30fps

In [95]:
initial = np.random.random(initial.shape)

In [96]:
shoulder_frame_id = arm_robot.model.getFrameId(SHOULDER)
elbow_frame_id = arm_robot.model.getFrameId(ELBOW)
wrist_frame_id = arm_robot.model.getFrameId(WRIST)

# Utility functions

In [97]:
def process_var(var, T, nq) -> Tuple[np.ndarray, np.ndarray]:
    # [q_1, q_2, ..., q_T, tauq_3, tauq_4, ..., tauq_T-2]

    # [ q1 , q2    q3  ,                       , qT-1, qT]
    # [ -    - , tauq_3, tauq_4, ..., tauq_T-2 ,  -   -  ]
    tq_unnormalized = var[:T * nq].reshape(T, nq) # tq =[q_1, q_2, ..., q_T]
    ttauq = var[T * nq:] # ttauq = [tauq_3, tauq_4, ..., tauq_T-2]


    # Get tq

    # Normalize shoulder
    shoulder_quaternion_unnormalized = tq_unnormalized[:, :4] # Shouler quaternion 4 premières valeurs de q

    shoulder_quaternion_norm = np.linalg.norm(
        shoulder_quaternion_unnormalized, 
        axis=1,
        keepdims=True
    )
    
    shoulder_quaternion_normalized = shoulder_quaternion_unnormalized / shoulder_quaternion_norm

    # Normalize elbow
    elbow_angle_unnormalized = tq_unnormalized[:, 4:].reshape(T, 2)

    elbow_angle_norm = np.linalg.norm(
        elbow_angle_unnormalized,
        axis=1,
        keepdims=True
    )

    elbow_angle_normalized = elbow_angle_unnormalized / elbow_angle_norm

    tq = np.concatenate(
        (
            shoulder_quaternion_normalized, 
            elbow_angle_normalized
        ), 
        axis=1
    )
    assert tq.shape == tq_unnormalized.shape

    return tq, ttauq

In [98]:
def get_velocity_acceleration(tq, T, nv, DT):

    tvq = np.empty((T - 2, nv))
    # nv=4

    # Pourquoi ces opérations ne sont pas vectorizés dans pinocchio ...
    for i in range(1, T - 1):
        tvq[i - 1] = pin.difference(arm_robot.model, tq[i - 1], tq[i + 1]) / (2 * DT)

    taq = (tvq[2:] - tvq[:-2]) / (2 * DT) 

    return tvq, taq

In [99]:
def get_3D_pose_velocity_acceleration(tq, T, DT):
    # [p_shoulder_1, p_elbow_1, p_wrist_1, ..., p_shoulder_T, p_elbow_T, p_wrist_T]

    tshoulder_p = np.empty((T, 3))
    telbow_p = np.empty((T, 3))
    twrist_p = np.empty((T, 3))

    for i in range(T):
        # Forward kinematics
        pin.framesForwardKinematics(arm_robot.model, arm_robot.data, tq[i])

        # Predicted 3D points
        tshoulder_p[i] = arm_robot.data.oMf[shoulder_frame_id].translation
        telbow_p[i] = arm_robot.data.oMf[elbow_frame_id].translation
        twrist_p[i] = arm_robot.data.oMf[wrist_frame_id].translation

    # Computes speed and acceleration
    tshoulder_v = (tshoulder_p[2:] - tshoulder_p[:-2]) / (2 * DT)
    telbow_v = (telbow_p[2:] - telbow_p[:-2]) / (2 * DT)
    twrist_v = (twrist_p[2:] - twrist_p[:-2]) / (2 * DT)

    tshoulder_a = (tshoulder_v[2:] - tshoulder_v[:-2]) / (2 * DT)
    telbow_a = (telbow_v[2:] - telbow_v[:-2]) / (2 * DT)
    twrist_a = (twrist_v[2:] - twrist_v[:-2]) / (2 * DT)

    data_pos3D = np.concatenate((tshoulder_p, telbow_p, twrist_p), axis=1).flatten()
    velocity = np.concatenate(
        (tshoulder_v, telbow_v, twrist_v), axis=0).flatten()
    acceleration = np.concatenate(
        (tshoulder_a, telbow_a, twrist_a), axis=0).flatten()
    return data_pos3D, velocity, acceleration

# Cost terms definitions

In [100]:
# If the Lagrange dynamics formulation cannote be totally satisfied,
# one can relax by minimizing the difference between:
# - the Lagrange dynamics torques tau_rec predicited from q, vq and aq.
# - the current predicted torques ttauq
# question: why not also going backward in time ? ici middle point

def full_body_dynamics(tq, tvq, taq, T, nv):
    
    ttau = np.empty((T - 4, nv))

    for i in range(2, T - 2):
        ttau[i - 2] = pin.rnea(arm_robot.model, arm_robot.data, tq[i], tvq[i - 1], taq[i - 2])

    return ttau.flatten()

In [101]:
# Build the cost function
def objective(var, observed_p, T, nq, nv ,debug=False) -> np.ndarray:
    tq, ttauq = process_var(var, T, nq)
    tvq, taq = get_velocity_acceleration(tq, T, nv, DT)
    tp, tv, ta = get_3D_pose_velocity_acceleration(tq, T, DT)
    ttau = full_body_dynamics(tq, tvq, taq, T, nv)


    if debug :
        print("Diff between 3D pose :", np.linalg.norm(observed_p - tp))
        print("Smooth velocity :", np.linalg.norm(tv))
        print("Smooth acceleration :", np.linalg.norm(ta))
        print("Smooth torque :", np.linalg.norm(ttauq))
        print("Dynamics :", np.linalg.norm(ttau - ttauq))

    res_p = observed_p - tp
    mask_p = np.abs(res_p) > 1
    res_p[mask_p] = 2 * np.sqrt(np.abs(res_p[mask_p])) - 1

    res = np.concatenate([
        1.5 * (res_p), 
        #0.2 * tv,
        #0.1 * ta,
        0.6 * ttauq,
        2 * (ttau - ttauq),
    ])

    return res

# Optimization

In [102]:
sol = least_squares(
    objective, 
    initial, 
    args=(ground_truth_p, T, arm_robot.model.nq, arm_robot.model.nv), 
    method='lm'
).x

In [103]:
sol_q, sol_tau = process_var(sol, T, arm_robot.model.nq)

In [104]:
for t in range(T):
    viz.display(sol_q[t])
    time.sleep(0.1)

In [105]:
for t in range(T):
    viz.display(q[t])
    time.sleep(1e-1)